# Assignment 7
- by Martim Silva 51304 and Alexandre Sobreira 59451

In [262]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [260]:
def lstm_func(nrEpochs, lag, X, y):

    ## LSTM Model
    lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer((lag,1)),    # The input layer will be lag
    tf.keras.layers.LSTM(10),  # Since we have an imput of 12 it doesnt make sence to use higher than that # trocar par DENSE
    tf.keras.layers.Dense(1, activation="tanh"), # Since I want to predict 1 value I want to output 1!
    ])
    
    ## Compile
    lstm_model.compile(optimizer = "Adam", loss = "mse")

    ##Adapt X
    X_lstm = np.expand_dims(X, axis = 2)

    ## 1st split
    print("Training with first 2400 and testing with 2401 to 2500")
    X_lstm_train = X_lstm[:2400]
    y_lstm_train = y[:2400]
    ### fit 
    lstm_model.fit(X_lstm_train, y_lstm_train, batch_size=1, epochs=nrEpochs, shuffle=False)
    ### pred
    X_lstm_pred = X_lstm[2401:2501]
    y_pred = lstm_model.predict(X_lstm_pred)
    print(mean_squared_error(y[2401:2501], y_pred))

    ## 2nd split and fit
    print("Training with 2401 to 4900 and testing with 4901 to 5000")
    lstm_model.fit(X_lstm[2401:4900], y[2401:4900], batch_size=1, epochs=nrEpochs, shuffle=False)
    ###pred
    y_pred = lstm_model.predict(X_lstm[4901:])
    print(mean_squared_error(y[4901:], y_pred))

In [276]:
def ftlfn_func(nrEpochs, lag, X, y): 
    
    ## FTLFN Model
    ftlf_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_dim=X.shape[1]),    
    tf.keras.layers.Dense(1),  
    ])
    
    ## Compile
    ftlf_model.compile(optimizer = "Adam", loss = "mse")

    ##Adapt X
    X = np.expand_dims(X, axis = 2)
    y = np.array(y)

    ## 1st split
    print("Training with first 2400 and testing with 2401 to 2500")
    X_fltf_train = X[:2400]
    y_fltf_train = y[:2400]
    ### fit 
    ftlf_model.fit(X_fltf_train, y_fltf_train, batch_size=1, epochs=nrEpochs, shuffle=False)
    ### pred
    X_fltf_pred = X[2401:2501]
    y_pred = ftlf_model.predict(X_fltf_pred)
    y_pred = np.array(y_pred)
    print(mean_squared_error(y[2401:2501], y_pred))


    ## 2nd split and fit
    print("Training with 2401 to 4900 and testing with 4901 to 5000")
    ftlf_model.fit(X[2401:4900], y[2401:4900], batch_size=1, epochs=nrEpochs, shuffle=False)
    ###pred
    y_pred = ftlf_model.predict(X[4901:])
    print(mean_squared_error(y[4901:], y_pred))



# Problem 1

In [272]:
# I need to use the sin function
# First I need to generate the x
periods = 100
lag = 12
points_per_period = 50
data = np.sin(np.linspace(0,2*np.pi*periods, num=periods*points_per_period)) #This is my x, it will be my angles, 0 to pi.
data                                                                         #it will creat equaly size spaces between 0 and .. (linspace)
                                                                            #and I apply the sin...
                                                                            # 
                                                                            # # lets add the lag


## Adapt Data for Time series 
X, y = [], []
for i in range(lag, data.shape[0]-1):
    X.append(data[i-lag:i]) # we getting 12 sample. My x will have 4000 samples each with 12 instances
    y.append(data[i])


X = np.array(X)
y = np.array(y)

 The sin function gave me the complete signal. But we need to adapt for this problem\
 To make predictions we need to to have several windows.\
 We consturct a data set  which has 12 datapoints and the label. Fazemos um shift de 12 em 12.\
 it says we need to use a filter of 12 (we are going to observe the 12 and predict the next 1). Then we shift again and so one

## i) no noise

FOr this type of problem, since we are comparing values, the MSE is the best lost function, for images would be the cross entropy

THe fit shuffles by default and we dont want that, we want to preserva the order. 

network needs to "warm up" to find the direction of the gradient. This will have  a huge impact on the MSE this  to pass the 

### LSTM

In [270]:
lstm_func(2, lag, X, y)

Training with first 2400 and testing with 2401 to 2500
Epoch 1/2
2400/2400 [==============================] - 13s 5ms/step - loss: 0.0637
Epoch 2/2
4/4 [==============================] - 0s 3ms/step
0.005339741994797782
Training with 2401 to 4900 and testing with 4901 to 5000
Epoch 1/2
2499/2499 [==============================] - 12s 5ms/step - loss: 0.0026
Epoch 2/2
3/3 [==============================] - 0s 4ms/step
0.001554377941643515


### FTLFN

In [277]:
ftlfn_func(2, lag, X, y)

Training with first 2400 and testing with 2401 to 2500
Epoch 1/2
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0460
Epoch 2/2
4/4 [==============================] - 0s 1ms/step
0.00045505211296543064


KeyError: 'val_loss'

## ii) with noise by adding a Gaussian N(0, 0.05).

In [ ]:
#add ao data o gaussian noise! (np.random(loc=0,scale (0.05, size = data[shape[0]])))
# I need to use the sin function
# First I need to generate the x
periods = 100
lag = 12
points_per_period = 50
data = np.sin(np.linspace(0,2*np.pi*periods, num=periods*points_per_period)) #This is my x, it will be my angles, 0 to pi.
                                                                         #it will creat equaly size spaces between 0 and .. (linspace)
                                                                            #and I apply the sin...
                                                                            # 
                                                                            # # lets add the lag"""
                                                                            
                                                                    
data = np.random.normal(loc=0, scale=0.05, size=data.shape[0])


X, y = [], []
for i in range(lag, data.shape[0]-1):
    X.append(data[i-lag:i]) # we getting 12 sample. My x will have 4000 samples each with 12 instances
    y.append(data[i])

X = np.array(X)
y = np.array(y)

### LSTM

In [ ]:
lstm_func(1, lag, X, y)

### FTLFN

In [ ]:
ftlfn_func(1, lag, X, y)

# Problem 2

In [ ]:
nrEpochs = 50

alpha_one = 0.6
alpha_two = -0.54
alpha_three = -0.44
data = [5,6,3]
nr_datapoints = 5000
X, y = [], []

#(np.random(loc=0,scale (0.01, size = data[shape[0]])))

for i in range(nr_datapoints-3):
    data.append(alpha_one*data[i+2]
    + alpha_two*data[i+1]
    + alpha_three*data[i])

for i in range(lag, len(data)-1):
    X.append(data[i-lag:i]) # we getting 12 sample. My x will have 4000 samples each with 12 instances
    y.append(data[i])

y = np.array(y)

## i) no noise, clean

### LSTM

In [ ]:
lstm_func(nrEpochs, lag, X, y)

### FTLFN

In [ ]:
ftlfn_func(nrEpochs, lag, X, y)

## ii) with noise by adding a Gaussian noise N(0, 0.01)

In [ ]:
# lets add the lag
alpha_one = 0.6
alpha_two = -0.54
alpha_three = -0.44
data = [5,6,3]
nr_datapoints = 5000

X, y = [], []

for i in range(nr_datapoints-3):
    data.append(alpha_one*data[i+2]
    + alpha_two*data[i+1]
    + alpha_three*data[i] + 0.01)

for i in range(lag, len(data)-1):
    X.append(data[i-lag:i]) # we getting 12 sample. My x will have 4000 samples each with 12 instances
    y.append(data[i])

y = np.array(y)

### LSTM

In [ ]:
lstm_func(nrEpochs, lag, X, y)

### FTLFN

In [ ]:
ftlfn_func(nrEpochs, lag, X, y)

## Final Remarks




to do
